In [757]:
import simpy
import csv
import pandas as pd
import numpy as np
import random

## `entities.py` ##

Took attributes that should be for BuiltCapital() / Residence()
Added self.money_to_rebuid, self.income
Changed self.insurance_coverage definition to point to household['Insurance']
Added self.residence
Delete io stuff and moved to io.py

In [758]:
# import pandas as pd
# import numpy as np

class Household(object):
    """Define a Household class with attributes and visualization methods.

        Methods have yet to be defined, but will likely include putting data into
        a pandas dataframe and having the ability to show recovery trajectories
        for each household
    """
    def __init__(self, simulation, household):

        # Household Attributes
        self.household = household
        self.name = household['Name']  # Name assigned to household
        self.income = household['Income'] # --+ added +--
        self.savings = household['Savings'] # --% Modified to read input data %--
        self.insurance = household['Insurance'] # --% Modified to read input data %--
        
        
        #Outputs
        self.story = [] # The story of events for each household
        self.inspection_put = 0.0 # Time put request in for house inspection
        self.inspection_get = 0.0 # Time get  house inspection
        self.claim_put = 0.0 # Time put request in for insurance settlement
        self.claim_get = 0.0 # Time get insurance claim settled
        self.claim_payout = 0.0 # Amount of insurance claim payout
        self.assistance_put = 0.0 # Time put request in for FEMA assistance
        self.assistance_get = 0.0 # Time get FEMA assistance
        self.assistance_request = 0.0 # Amount of money requested from FEMA
        self.assistance_payout = 0.0 # Amount of assistance provided by FEMA
        self.money_to_rebuild = self.savings # Total funds available to household to rebuild house
        self.house_put = 0.0 # Time put request in for house rebuild
        self.house_get = 0.0 # Time get house rebuild completed
        self.loan_put = 0.0 # Time put request for loan
        self.loan_get = 0.0 # Time get requested loan
        self.loan_amount = 0.0 # Amount of loan
        self.permit_put = 0.0 # Time put request for building permit
        self.permit_get = 0.0 # Time get requested building permit
        
        # Function calls
        self.residence = Residence(simulation, household) # Assign residence to the household
        self.setStory() # Start stories with non-disaster attributes
    
    def setStory(self):
        
        # Start stories with non-disaster attributes
        self.story.append(
        '{0} lives in a {1} bedroom {2} ({3}). '.format(self.name, 
                                                        self.residence.bedrooms, 
                                                        self.residence.occupancy,
                                                        self.residence.address
                                                        )
        )

    def story_to_text(self): # --% modified %--
        '''
            Function to join the strings within the story list created during DESaster processes
        '''
        
        return ''.join(self.story)


## `capitals.py` ##

Call to Containers() was missing arguments. If only two is given with no keywords, second argument is assumed to be the capacity, not the initial level. To have capacity default to infinit and set an initial level, need to use "init" keyword. Changed to:

`simpy.Container(simulation, init=financial_cap_data['fema aid'])`

In [759]:
# import simpy
# import csv

class HumanCapital(): # --% created a separate class for just human capitals %--
    """This is the main resource data structure.
    
        Provide a data dictionary. The dictionary MUST include all the
        resources listed below. 
    """
    def __init__(self, simulation, human_cap_data): # --% changed argument name %--
                
        self.data = human_cap_data
        
        ## HUMAN CAPITALS ## --% changed to make reference to human_cap_data %--
        
        try:
            self.inspectors = simpy.Resource(simulation, human_cap_data['inspectors'])
            self.insurance_adjusters = simpy.Resource(simulation, human_cap_data['insurance adjusters'])
            self.fema_processors = simpy.Resource(simulation, human_cap_data['fema processors'])
            self.permit_processors = simpy.Resource(simulation, human_cap_data['permit processors'])
            self.contractors = simpy.Resource(simulation, human_cap_data['contractors'])
            self.loan_processors = simpy.Resource(simulation, human_cap_data['loan processors'])
            self.engineers = simpy.Resource(simulation, human_cap_data['engineers'])
        except ValueError as e:
            
            print ("You are missing a config value, or your value is zero. All" 
                    "values must have a positive number, see error: {0}".format(e))
        except KeyError as f:
            print ("You are missing {0} as a capital. Please re-add in your"
                    " config file".format(f))
                    
        except Exception as g:
            print ("Something went really wrong, capitals failed to load."
                    " See error {0}".format(g))

class FinancialCapital(): # --% created a separate class for just financial capitals %--
        def __init__(self, simulation, financial_cap_data): # --% changed/added arguments %--
            
            ## FINANCIAL CAPITALS ##  --% changed to make reference to human_cap_data %--
            self.fema_aid = simpy.Container(simulation, init=financial_cap_data['fema aid'])
        
class BuiltCapital(): # --% created a separate class for just financial capitals %--
        def __init__(self, simulation, asset):
            self.setYearBuilt(asset)
            self.setValue(asset)
            self.setDamageState(asset)  
            self.setDamageValue(asset) 
            self.setInspection(simulation, asset)
        def setYearBuilt(self, asset):
            self.age = asset['Year Built']
        def setValue(self, asset):
            self.value = asset['Value']
        def setDamageState(self, asset):
            self.damage_state = asset['Damage State']
        def setDamageValue(self, asset):
            self.damage_value = asset['Damage Value']
        def setInspection(self, simulation, asset):
            self.inspection = simulation.event()
            self.inspect_start = None
            self.inspect_stop = None

class Building(BuiltCapital):
    def __init__(self, simulation, building):
        self.setAddress(building)
#         self.setResident(building)
        try:
            self.setOccupancy(building)
        except AttributeError:
            print('Invalid occupancy type provided: {0}.'.format(building['occupancy']))
        self.setMonthlyCost(building)
        self.setFloors(building)
        self.setYearBuilt(asset)
        self.setValue(building)
        self.setDamageState(building)  
        self.setDamageValue(building) 
        self.setInspection(simulation, building)
    def setAddress(self, building):
        self.address = building['Address']
    def setOccupancy(self, building):
        self.occupancy = building['Occupancy']
    def setMonthlyCost(self, building):
        self.cost = building['Cost']
    def setBuildingArea(self, building):
        self.cost = building['Area']
    def setFloors(self, building):
        self.floors = building['Floors']
        
class Residence(Building):
    #Can verify that occupancy types only relate to residences
    def __init__(self, simulation, residence):
        self.setAddress(residence)
        self.setBuildingArea(residence)
        self.setMonthlyCost(residence)
        self.setFloors(residence)
        self.setOccupancy(residence)
        self.setBedrooms(residence)
        self.setBathrooms(residence)
        self.setYearBuilt(residence)
        self.setValue(residence)
        self.setDamageState(residence)  
        self.setDamageValue(residence) 
        self.setInspection(simulation, residence)
    def setOccupancy(self, residence):
        if residence['Occupancy'] in ('House', 'Apartment', 'Condo'):
            self.occupancy = residence['Occupancy']
        else:
            raise AttributeError(residence['Occupancy'])
    def setBedrooms(self, residence):
        self.bedrooms = residence['Bedrooms']
    def setBathrooms(self, residence):
        self.bathrooms = residence['Bathrooms']

### Stuff to delete from old file
# class House:
#     def __init__(self, inp):
#         self.number = inp[0]
#         self.lat = inp[2]
#         self.long = inp[1]
        
#     def get_capitals_list(self):
#         c_list = []
#         for i in dir(self):
#             if "__" not in i:
#                 c_list.append(i)
#         return c_list
        
#     def get_capitals_values(self):
#         return self.data
    
# --% Deleted. Households are not capitals. They are entities. Moved attributes to Residence() %-- 
# --% Need to separate out i/o. %--     
#     def load_households(self, simulation): #this is only used internally
#         #TODO load all input files from the IO file
#         input_file = "../inputs/housing_stock.csv"
#         stock = []
#         with open(input_file) as file:
#             reader = csv.reader(file)
#             for i in reader:
#                 stock.append(House(i)) #House is a class defined below
#         stock.pop(0)
#         housing_stock = simpy.Store(simulation)
#         for i in stock:
#             housing_stock.put(i)
        
#         return housing_stock
            


## `requests.py` ##

Changed sequence of arguments in the function definitions to be simulation, capitals, entity, callbacks)

Started using self.money_to_rebuild to track money

In [760]:
# from desaster.config import inspection_time, adjuster_time, fema_process_time
# from desaster.config import engineering_assessment_time, loan_process_time
# from desaster.config import permit_process_time

def inspection(simulation, human_capital, entity, callbacks = None):
    """Request an inspection, do inspection, update entity attribute times.

    Keyword Arguments:
    entity -- An entity object from the Entity() class. Must have a value for
              attribute 'insurance_coverage', which should be set at __init__()

    simulation -- A simpy.Environment() object. This references the simulation
                  environment, and is usually set as the first variable in a
                  simulation, e.g. simulation = simpy.Environment().

    callbacks -- a generator function containing any processes you want to start
                 after the completion of the insurance claim. If this does not
                 contain a yield (therefore isn't a generator), simpy will throw
                 an error. Defaults to None.

    Returns or Attribute Changes:

    entity.inspection_put -- Record time of inspection request
    entity.inspection_get -- Record time of inspection completion
    entity.story -- append natural language summary of process
    """

    with human_capital.inspectors.request() as request:

        # Put in request for an inspector (shared resource)
        entity.inspection_put = simulation.now
        #the inspection_put attribute can be added after init,
        yield request

        # Duration of inspection
        yield simulation.timeout(inspection_time)

        # The time that the inspection has been completed
        entity.inspection_get = simulation.now
        response_time = simulation.now
        #write their story
        entity.story.append(
        "{1}'s house was inspected {0} days after the event. ".format(response_time, entity.name))
    
    # Start stories with non-disaster attributes
    entity.story.append(
                    'The event caused ${0} of damage to the residence. '.format(entity.residence.damage_value)
                    )

    if callbacks is not None:
        yield simulation.process(callbacks)
    else:
        pass

def insurance_claim(simulation, human_capital, entity, callbacks = None):
    """File an insurance claim, assign claim amounts to entity objects.

    Keyword arguments:
    entity -- An entity object from the Entity() class. Must have a value for
              attribute 'insurance_coverage', which should be set at __init__()

    simulation -- A simpy.Environment() object. This references the simulation
                  environment, and is usually set as the first variable in a
                  simulation, e.g. simulation = simpy.Environment().

    callbacks -- a generator function containing any processes you want to start
                 after the completion of the insurance claim. If this does not
                 contain a yield (therefore isn't a generator), simpy will throw
                 an error. Defaults to None.

    Returns or attribute changes:

    entity.claim_put -- Record current simulation time at the time the entity
                        enters the adjuster queue

    entity.claim_payout -- Set claim payout equal to coverage amount

    entity.claim_get -- Record simulation time when entity recieves payout

    entity.story -- Append natural language sentences to entities story.
    """

    with human_capital.insurance_adjusters.request() as request:
        # Record time that claim is put in
        entity.claim_put = simulation.now
        yield request

        # Duration of claim processing
        yield simulation.timeout(adjuster_time)

        # --% Set insurance payout equal to damage value for simplicity %--
        entity.claim_payout = entity.residence.damage_value
        entity.money_to_rebuild += entity.claim_payout

        # Record when the time when household gets claim payout
        entity.claim_get = simulation.now

    # Write the household's story
    entity.story.append(
        '{0} received a ${1} insurance payout {2} days after the event. '.format(
        entity.name, entity.claim_payout, entity.claim_get))
    
    if callbacks is not None:
        yield simulation.process(callbacks)

    else:
        pass

def fema_assistance(simulation, human_capital, financial_capital, entity, callbacks = None):
    """Request and receive assistance from fema.

    entity -- An entity object from the Entity() class. Must have a value for
              attributes: 'residence.damage_value', 'claim_payout',
              which should be set at __init__()

    simulation -- A simpy.Environment() object. This references the simulation
                  environment, and is usually set as the first variable in a
                  simulation, e.g. simulation = simpy.Environment().

    callbacks -- a generator function containing any processes you want to start
                 after the completion of the insurance claim. If this does not
                 contain a yield (therefore isn't a generator), simpy will throw
                 an error. Defaults to None.

    function version date: Jan 25 2015

    Returns or Attribute Changes:

    entity.assistance_put -- Records sim time of fema processor request

    entity.assistance_get -- Records sim time of fema assistance reciept

    entity.assistance_request -- The amount of money being requested by the
                                 entity, equal to the residence.damage_value minus the
                                 claim_payout provided by the insurance process

    entity.assistance_payout -- amount of money given to the entity, equal to
                                the request amount, or whatever is left in the
                                financial_capital.fema_aid container, whichever is higher.

    """
    
    # --% Redid logic so those that don't need FEMA assistance don't wait for FEMA processors %--

    # To process assistance request must request and wait
    #for a FEMA application processor
    if entity.money_to_rebuild >= entity.residence.damage_value:
        pass
    else:
        # Put in request for FEMA individual assistance; record time requested
        entity.assistance_put = simulation.now
        yield human_capital.fema_processors.request()

        # Time required for FEMA to process assistance request
        yield simulation.timeout(fema_process_time)

        # Record time household gets FEMA assistance
        entity.assistance_get = simulation.now

        # Compute amount of assistance requested from FEMA; if insurance payout covers repair cost it is zero
        entity.assistance_request = entity.residence.damage_value - entity.claim_payout


        # If requesting assistance, determine if FEMA has money left to provide assistance
        ## I think this should actually just be a request to the resource and if
        ## there isn't money left, it just sits in the queue.


        if entity.assistance_request <= financial_capital.fema_aid.level:

            entity.assistance_payout = entity.assistance_request
            entity.money_to_rebuild += entity.assistance_payout

            # Write the household's story
            entity.story.append(
                '{0} received ${1} from FEMA {2} days after the event. '.format(
                entity.name, entity.assistance_payout, entity.assistance_get))

            yield financial_capital.fema_aid.get(entity.assistance_request)

        elif financial_capital.fema_aid.level > 0:

            entity.assistance_payout = financial_capital.fema_aid.level
            entity.money_to_rebuild += entity.assistance_payout

            # Write the household's story
            entity.story.append(
             '{0} requested ${1} from FEMA but only received ${2}. '
             .format(entity.name, entity.assistance_request, entity.assistance_payout))

            entity.story.append(
                'They received the assistance {0} days after the event. '.format(
                entity.assistance_get))

            yield financial_capital.fema_aid.get(financial_capital.fema_aid.level)

        else:
            entity.assistance_payout = 0.0

            # Write the household's story
            entity.story.append(
            '{0} received no money from FEMA because of inadequate funding. '
            .format(entity.name))

    if callbacks is not None:
        yield simulation.process(callbacks)

    else:
        pass

def engineering_assessment(simulation, human_capital, entity, callbacks = None):
    """Request an engineering assessment"""
    with human_capital.engineers.request() as request:
        entity.assessment_put = simulation.now #time of request
        yield request

        yield simulation.timeout(engineering_assessment_time)
        entity.assessment_get = simulation.now #when assessment is received

    entity.story.append(
    '{0} received an engineering assessment {1} days after the event. '
    .format(entity.name, entity.assessment_get))

    if callbacks is not None:
        yield simulation.process(callbacks)
    else:
        pass

def loan(simulation, human_capital, entity, callbacks = None):
    # --% Need to make sure people who don't need to apply for a loan don't wait for loan processing %--
    if entity.money_to_rebuild < entity.residence.damage_value:
        entity.loan_put = simulation.now
        yield human_capital.loan_processors.request()

        yield simulation.timeout(loan_process_time)
        entity.loan_get = simulation.now

        # --% Added initial code to produce loan payout %--
        # TODO Need code here to determine how much money the entity requests for their
        #loan and whether the are approved
        # TODO make sure to add to entity.money_to_rebuild
        
       
        entity.loan_amount = entity.residence.damage_value - entity.claim_payout - entity.assistance_payout
#             entity.money_to_rebuild += entity.loan_amount

        entity.story.append(
        "{0} received a loan for ${1} {2} days after the event. "
        .format(entity.name, entity.loan_amount, entity.loan_get))

    if callbacks is not None:
        yield simulation.process(callbacks)
    else:
        pass

def permit(simulation, human_capital, entity, callbacks = None):

    # --% Added logic to prevent a permit being issued if an engineering assessment wasn't obtained. %--
    if entity.assessment_get > 0.0:
        """Request a permit for building."""
        with human_capital.permit_processors.request() as request:
            entity.permit_put = simulation.now
            yield request

            yield simulation.timeout(permit_process_time)
            entity.permit_get = simulation.now

        entity.story.append(
        "{0} received permit approval {1} days after the event. "
        .format(entity.name, entity.permit_get))
    else:
        entity.story.append(
        "{0} could not get the necessary engineering assessment to apply for a permit. "
        .format(entity.name))
    
    if callbacks is not None:
        yield simulation.process(callbacks)
    else:
        pass

## `rebuild.py` ##

Made argument refer to human_capital, rather than directly to contractors, to be consistent with other functions.

In [761]:
# from desaster.config import sfr_rebuild_time

def rebuild_house(simulation, human_capital, entity): 
        
        if entity.permit_get > 0.0 and entity.money_to_rebuild >= entity.residence.damage_value:
            with human_capital.contractors.request() as request:
                # Put in request for contractors to repair house
                entity.house_put = simulation.now
                yield request

                # Time required to rebuild house
                yield simulation.timeout(sfr_rebuild_time)

                # Record time when household gets house
                entity.house_get = simulation.now

            # Write the household's story
            entity.story.append(
                'The house was rebuilt {0} days after the event, taking {1} days to rebuild. '.format(
                entity.house_get, sfr_rebuild_time))
        elif entity.money_to_rebuild < entity.residence.damage_value:
            entity.story.append(
                '{0} was unable to get enough money to rebuild. '.format(
                entity.name))
        else: 
            entity.story.append(
                '{0} was unable to get a permit to rebuild. '.format(
                entity.name))

## `search.py` ##

## `config.py` ##

In [762]:
#import random

random.seed(69)

inspection_mean = 10.0
inspection_std = 0
inspection_time = abs(random.gauss(inspection_mean, inspection_std))

adjuster_mean = 30.0
adjuster_std = 0.0
adjuster_time = abs(random.gauss(adjuster_mean, adjuster_std))

fema_process_mean = 30.0
fema_process_std = 0.0
fema_process_time = abs(random.gauss(fema_process_mean, fema_process_std))

engineering_mean = 7.0
engineering_std = 0.0
engineering_assessment_time = abs(random.gauss(engineering_mean, engineering_std))

loan_process_mean = 30.0
loan_process_std = 0.0
loan_process_time = abs(random.gauss(loan_process_mean, loan_process_std))

permit_process_mean = 30.0
permit_process_std = 0.0
permit_process_time = abs(random.gauss(permit_process_mean, permit_process_std))

sfr_rebuild_mean = 30.0
sfr_rebuild_std = 0.0
sfr_rebuild_time = abs(random.gauss(sfr_rebuild_mean, sfr_rebuild_std))

## `io.py` ##

In [763]:
# from desaster import config


def view_config():
    var = vars(config)
    conf = dir(config)
    result = {}
    for i in conf:
        if "__" not in i and "random" not in i:
            print ("{} = {}".format(i, var[i]))
            #print (i)
            result[i] = var[i]
            
        else:
            pass
    return result

def imp_concat():
    input1 = pd.read_csv("../inputs/Work out of County Households.csv")
    input2 = pd.read_csv("../inputs/Work in County Households.csv")
    input3 = pd.read_csv("../inputs/Middle High School Households.csv")
    input4 = pd.read_csv("../inputs/Elementary School Households.csv")
    input5 = pd.read_csv("../inputs/Grocery Households.csv")
    all_inputs = [input1, input2, input3, input4, input5]
    data = pd.concat(all_total)
    return data

def get_names():
    lasts = pd.read_csv("../inputs/last_names.csv")
    data = lasts.iloc[0:1366]
    return data['name']

## Simulation Testing ##

In [764]:
simulation = simpy.Environment()

When packaging need to put back references to request. etc. and rebuild. etc.

Changed function name to master_process  ...having it "simulation" is confusing

Changed arguments so that there are separate variables for the different capitals

In [765]:
def find_rebuild_money(simulation, human_capital, financial_capital, entity):
    if entity.insurance == 1:
        yield simulation.process(insurance_claim(simulation, human_capital, entity))
    
    yield simulation.process(fema_assistance(simulation, human_capital, financial_capital, entity)) 
           
    yield simulation.process(loan(simulation, human_capital, entity))
    
    return 'Exhausted funding search'



In [766]:
def master_process(simulation, human_capital, financial_capital, entity):
       
    yield simulation.process(inspection(simulation, human_capital, entity))

    money_to_rebuild_patience = simulation.timeout(100, value='Gave up')
    
    rebuild_money_search = simulation.process(find_rebuild_money(simulation, human_capital, financial_capital, entity))
    
    
    money_search_outcome = yield money_to_rebuild_patience | rebuild_money_search

    if money_search_outcome == {money_to_rebuild_patience: 'Gave up'}:
        entity.story.append(
                            '{0} gave up searching for money to rebuild. '.format(
                            entity.name))
        pass
    
    if money_search_outcome == {rebuild_money_search: 'Exhausted funding search'} and \
    entity.money_to_rebuild >= entity.residence.damage_value:
        yield simulation.process(engineering_assessment(simulation, human_capital, entity))
        
        yield simulation.process(permit(simulation, human_capital, entity))
        
        yield simulation.process(rebuild_house(simulation, human_capital, entity))
   
    if money_search_outcome == {rebuild_money_search: 'Exhausted funding search'} and \
        entity.money_to_rebuild < entity.residence.damage_value:
            entity.story.append(
                                '{0} was unable to find enough money to rebuild. '.format(
                                entity.name))
            pass

Now must separate define human_cap_data and financial_cap_data ....previously just "data"

In [767]:
human_cap_data = {
        "inspectors": 1,
        "insurance adjusters": 1,
        "fema processors": 1,
        "permit processors": 1,
        "contractors": 1,
        "loan processors": 1,
        "engineers": 1
                 }

In [768]:
financial_cap_data = {"fema aid": 35000}
# financial_cap_data = {"fema aid": 0}

No longer make call to Capitals() becuase make separate classes for each type %--

Formerly: `res = desaster.Capitals(sim, human_cap_data) # Need reference to desaster. for packages`

In [769]:
human_capital = HumanCapital(simulation, human_cap_data) 

Now also need to make call to FinancialCapital (e.g., for fema aid)

In [770]:
financial_capital = FinancialCapital(simulation, financial_cap_data) 

Read in household data. This data file contains attributes for the household *AND* the household's residence. At the moment this prevents a household from owning multiple residences, which will require using separate tables and doing a lookup to assign properties.

In [771]:
file_path = "../inputs/household_inputs_test.csv"
households_df = pd.read_csv(file_path)
households_df.head()

,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Floors,Year Built,Value,Damage State,Damage Value
0,Alfred,30000,10000,0,62 That St,House,100,1,1,700,1,1920,100000,None,20000
1,Bruce,100000,1000000,1,720 This Rd,House,100000,6,5,5000,3,1920,10000000,Slight,50000
2,Selena,10000,100,0,1001 Other Ave,Apartment,10,0,1,250,1,1960,10000,Complete,8000
3,Fish,50000,1000,1,26000 Out There Lane,Condo,2000,4,2,2000,2,2010,800000,Moderate,650000


Instantiate household objects from Household()

When packaging must reference Household() as entities.Household()

In [772]:
household = {}
for i in households_df.index:
    household[i] = Household(simulation, households_df.iloc[i])

In [773]:
for i in range(len(household)):
    #print (household[i])
    simulation.process(master_process(simulation, human_capital, financial_capital, household[i]))

In [774]:
simulation.run()

In [778]:
household[3].story_to_text()

"Fish lives in a 4 bedroom Condo (26000 Out There Lane). Fish's house was inspected 40.0 days after the event. The event caused $650000 of damage to the residence. Fish received a $650000 insurance payout 80.0 days after the event. Fish received an engineering assessment 87.0 days after the event. Fish received permit approval 137.0 days after the event. The house was rebuilt 167.0 days after the event, taking 30.0 days to rebuild. "